In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import time

count = 0
thisMonthFinancialStatements = '2022Q4'
errorID = []

def 連接到資料庫(db):
    global sql,conn,cursor,ids
    # 連接到資料庫A ====================================================================
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    sql = '''select * from financialStatements t'''

    # 連接到資料庫B
    sqlStock = '''select * from stock t'''
    dfStock = pd.read_sql(sqlStock,conn)
    dfStock = pd.DataFrame(dfStock)
    ids = dfStock['code'][count:]
    # ===================================================================================

import random

def 導入資料庫(dataExport2):
    for index, row in dataExport2.iterrows():
        try:
            cursor.execute('INSERT INTO financialStatements values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', row)
            conn.commit()
        except:
            pass

def 爬蟲(count,id):
    try:
        dataExport = pd.DataFrame()
        selected_cols = ['代碼','季度','營業毛利率','營業利益率','稅前淨利率','稅後淨利率','稅後淨利率(母公司)',
    '每股稅前盈餘(元)','每股稅後盈餘(元)','每股淨值(元)','股東權益報酬率(季累計)','股東權益報酬率(年預估)','資產報酬率(季累計)',
    '資產報酬率(年預估)','營收年成長率','毛利年成長率','營業利益年成長率','稅前淨利年成長率','稅後淨利年成長率','稅後淨利年成長率(母公司)',
    '每股稅後盈餘年成長率','現金(%)','應收帳款(%)','存貨(%)','速動資產(%)','流動資產(%)','基金與投資(%)','固定資產(%)',
    '無形資產(%)','其他資產(%)','現金季成長率','應收帳款季成長率','存貨季成長率','流動資產季成長率','基金與投資季成長率',
    '固定資產季成長率','無形資產季成長率','其他資產季成長率','資產總額季成長率','現金年成長率','應收帳款年成長率',
    '存貨年成長率','流動資產年成長率','基金與投資年成長率','固定資產年成長率','無形資產年成長率','其他資產年成長率',
    '資產總額年成長率','應付帳款(%)','流動負債(%)','長期負債(%)','其他負債(%)','負債總額(%)','普通股股本(%)','股東權益總額(%)',
    '應付帳款季成長率','流動負債季成長率','長期負債季成長率','其他負債季成長率','負債總額季成長率','普通股股本季成長率',
    '股東權益總額季成長率','應付帳款年成長率','流動負債年成長率','長期負債年成長率','其他負債年成長率','負債總額年成長率',
    '普通股股本年成長率','股東權益總額年成長率','現金比','速動比','流動比','利息保障倍數','現金流量比(季累計)',
    '現金流量比(年預估)','營業成本率','營業費用率','應收帳款週轉率(次/年)','應收款項收現日數(日)','應付帳款週轉率(次/年)',
    '應付款項付現日數(日)','存貨週轉率(次/年)','平均售貨日數(日)','固定資產週轉率(次/年)','總資產週轉率(次/年)',
    '淨值週轉率(次/年)','應收帳款佔營收比率(季累計)','應收帳款佔營收比率(年預估)','存貨佔營收比率(季累計)',
    '存貨佔營收比率(年預估)','每股營業現金流量(元)','每股投資現金流量(元)','每股融資現金流量(元)','每股淨現金流量(元)',
    '每股自由現金流量(元)','負債對淨值比率','長期資金適合率','所得稅佔稅前淨利比率','業外損益佔營收比率',
    '業外損益佔稅前淨利比率','財報評分(100為滿分)']
        dataExport = pd.DataFrame(columns=selected_cols)
        
        url = f'https://goodinfo.tw/tw/StockFinDetail.asp?RPT_CAT=XX_M_QUAR_ACC&STOCK_ID={id}'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.54'}
        res = requests.get(url, headers = headers, timeout=25)
        res.encoding = "utf-8"
        
        soup = BeautifulSoup(res.text,"lxml") 
        data = soup.select_one("#txtFinBody")

        time.sleep(10)
            
        # 【重整表格】 ====================================================================
        dfs = pd.read_html(data.prettify())
        df = dfs[1]
        df2 = df

        df2_columns = df2.columns
        df2_columns = ['季度' if x == '獲利能力' else x for x in df2_columns]

        # 将列名添加为新的一行
        df2.loc[-1] = df2_columns

        # 将索引值加一，将新行移动到第一行
        df2.index = df2.index + 1
        df2 = df2.sort_index()
        df2 = df2.reset_index(drop=True)

        # 修改columns名稱，先計算總共有幾列
        z = ['index']
        for x in range(len(df2.columns)-1):
            z.append(x)
        df2.columns = z

        # Transpose DataFrame
        df3 = df2.copy()
        df3.set_index('index',inplace=True)
        df3 = df3.T
        df3.insert(0,'代碼',id)

        # 整理表格
        try:
            df4 = df3.drop(columns=['獲利年成長率','各項資產佔總資產比重','資產季成長率','資產年成長率','負債&股東權益佔總資產','負債&股東權益季增減率','負債&股東權益年增減率','償債能力','經營能力','現金流量狀況','其他指標'])
        except:
            df4 = df3.drop(columns=['獲利年成長率','各項資產佔總資產比重','資產季成長率','資產年成長率','負債&股東權益佔總資產','負債&股東權益季增減率','負債&股東權益年增減率','經營能力','現金流量狀況','其他指標'])
        df4.columns = df4.columns.str.replace(' ', '').str.replace('  ','')
        
        # 整理並輸出
        dataExport2 = pd.concat([dataExport, df4], axis=0, ignore_index=True)
        dataExport2 = dataExport2.fillna(0)
        
        # 只选择特定列，并填充缺失值为 0
        dataExport2 = dataExport2.loc[:, selected_cols].fillna(0)

        導入資料庫(dataExport2)
    except:
        print(f'{count}没有找到{id}资料')
        time.sleep(int(format(random.randint(15,25))))

    
from tqdm import tqdm

def 导入季资料(count,thisMonthFinancialStatements):
    progress_bar = tqdm(ids,desc="正在执行第{}次".format(0))

    for id in progress_bar:
        # 先查詢本地資料庫
        dfdatabase = pd.read_sql(sql, conn)
        dfdatabaseMask = dfdatabase['代碼'] == id
        dfdatabase2 = dfdatabase[dfdatabaseMask]
        
        if thisMonthFinancialStatements not in dfdatabase2['季度'].values:
            爬蟲(count,id)

            progress_bar.set_description("正在执行第{}次".format(count+1))
            progress_bar.update(1)

            count+=1
            time.sleep(int(format(random.randint(15,25))))
        else:
            progress_bar.set_description("正在执行第{}次".format(count+1))
            progress_bar.update(1)

            count+=1
            pass

    progress_bar.close()

if __name__ == "__main__":
    連接到資料庫('goodinfoRevenue.db')
    导入季资料(count,thisMonthFinancialStatements)
     
    print('爬蟲完畢')
    conn.close()

正在执行第220次:  14%|█▎        | 241/1753 [09:03<45:10,  1.79s/it]  

220没有找到1802资料


正在执行第224次:  14%|█▍        | 245/1753 [09:45<2:26:18,  5.82s/it]

224没有找到1817资料


正在执行第236次:  15%|█▍        | 258/1753 [10:45<35:43,  1.43s/it]  

236没有找到2009资料


正在执行第1753次: 100%|██████████| 1753/1753 [56:49<00:00,  1.95s/it]

爬蟲完畢


In [8]:
連接到資料庫('goodinfoRevenue.db')

def 爬蟲(count,id):
    try:
        dataExport = pd.DataFrame()
        selected_cols = ['代碼','季度','營業毛利率','營業利益率','稅前淨利率','稅後淨利率','稅後淨利率(母公司)',
    '每股稅前盈餘(元)','每股稅後盈餘(元)','每股淨值(元)','股東權益報酬率(季累計)','股東權益報酬率(年預估)','資產報酬率(季累計)',
    '資產報酬率(年預估)','營收年成長率','毛利年成長率','營業利益年成長率','稅前淨利年成長率','稅後淨利年成長率','稅後淨利年成長率(母公司)',
    '每股稅後盈餘年成長率','現金(%)','應收帳款(%)','存貨(%)','速動資產(%)','流動資產(%)','基金與投資(%)','固定資產(%)',
    '無形資產(%)','其他資產(%)','現金季成長率','應收帳款季成長率','存貨季成長率','流動資產季成長率','基金與投資季成長率',
    '固定資產季成長率','無形資產季成長率','其他資產季成長率','資產總額季成長率','現金年成長率','應收帳款年成長率',
    '存貨年成長率','流動資產年成長率','基金與投資年成長率','固定資產年成長率','無形資產年成長率','其他資產年成長率',
    '資產總額年成長率','應付帳款(%)','流動負債(%)','長期負債(%)','其他負債(%)','負債總額(%)','普通股股本(%)','股東權益總額(%)',
    '應付帳款季成長率','流動負債季成長率','長期負債季成長率','其他負債季成長率','負債總額季成長率','普通股股本季成長率',
    '股東權益總額季成長率','應付帳款年成長率','流動負債年成長率','長期負債年成長率','其他負債年成長率','負債總額年成長率',
    '普通股股本年成長率','股東權益總額年成長率','現金比','速動比','流動比','利息保障倍數','現金流量比(季累計)',
    '現金流量比(年預估)','營業成本率','營業費用率','應收帳款週轉率(次/年)','應收款項收現日數(日)','應付帳款週轉率(次/年)',
    '應付款項付現日數(日)','存貨週轉率(次/年)','平均售貨日數(日)','固定資產週轉率(次/年)','總資產週轉率(次/年)',
    '淨值週轉率(次/年)','應收帳款佔營收比率(季累計)','應收帳款佔營收比率(年預估)','存貨佔營收比率(季累計)',
    '存貨佔營收比率(年預估)','每股營業現金流量(元)','每股投資現金流量(元)','每股融資現金流量(元)','每股淨現金流量(元)',
    '每股自由現金流量(元)','負債對淨值比率','長期資金適合率','所得稅佔稅前淨利比率','業外損益佔營收比率',
    '業外損益佔稅前淨利比率','財報評分(100為滿分)']
        dataExport = pd.DataFrame(columns=selected_cols)
        
        url = f'https://goodinfo.tw/tw/StockFinDetail.asp?RPT_CAT=XX_M_QUAR_ACC&STOCK_ID={id}'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.54'}
        res = requests.get(url, headers = headers, timeout=25)
        res.encoding = "utf-8"
        
        soup = BeautifulSoup(res.text,"lxml") 
        data = soup.select_one("#txtFinBody")

        time.sleep(10)
            
        # 【重整表格】 ====================================================================
        dfs = pd.read_html(data.prettify())
        df = dfs[1]
        df2 = df

        df2_columns = df2.columns
        df2_columns = ['季度' if x == '獲利能力' else x for x in df2_columns]

        # 将列名添加为新的一行
        df2.loc[-1] = df2_columns

        # 将索引值加一，将新行移动到第一行
        df2.index = df2.index + 1
        df2 = df2.sort_index()
        df2 = df2.reset_index(drop=True)

        # 修改columns名稱，先計算總共有幾列
        z = ['index']
        for x in range(len(df2.columns)-1):
            z.append(x)
        df2.columns = z

        # Transpose DataFrame
        df3 = df2.copy()
        df3.set_index('index',inplace=True)
        df3 = df3.T
        df3.insert(0,'代碼',id)

        # 整理表格
        try:
            df4 = df3.drop(columns=['獲利年成長率','各項資產佔總資產比重','資產季成長率','資產年成長率','負債&股東權益佔總資產','負債&股東權益季增減率','負債&股東權益年增減率','償債能力','經營能力','現金流量狀況','其他指標'])
        except:
            df4 = df3.drop(columns=['獲利年成長率','各項資產佔總資產比重','資產季成長率','資產年成長率','負債&股東權益佔總資產','負債&股東權益季增減率','負債&股東權益年增減率','經營能力','現金流量狀況','其他指標'])
        df4.columns = df4.columns.str.replace(' ', '').str.replace('  ','')
        
        # 整理並輸出
        dataExport2 = pd.concat([dataExport, df4], axis=0, ignore_index=True)
        dataExport2 = dataExport2.fillna(0)
        
        # 只选择特定列，并填充缺失值为 0
        dataExport2 = dataExport2.loc[:, selected_cols].fillna(0)

        導入資料庫(dataExport2)
    except:
        print(f'{count}没有找到{id}资料')
        time.sleep(int(format(random.randint(15,25))))



爬蟲(202,'2009')
conn.close()